# 2 - Prompting Best Practices with Amazon Nova Models

The effectiveness of prompts is contingent upon the quality of the information provided and the craftsmanship of the prompt itself. Prompts may encompass instructions, questions, contextual details, inputs, and examples to effectively guide the model and enhance the quality of the results. This document outlines strategies and tactics for optimizing the performance of Amazon Nova Family of Models. The methods presented herein may be employed in various combinations to amplify their effectiveness. We encourage users to engage in experimentation to identify the approaches most suitable for their specific needs. More details on prompt best practices can be found [here](https://docs.aws.amazon.com/nova/latest/userguide/prompting-text-understanding.html)

## Before Starting Prompt Engineering

Before starting prompt engineering, it is highly recommended to have following elements in place, so you can iteratively develop the most optimal prompt for your use case:

1. **Define your use case**: Define your use case you want to achieve on 4 dimensions
    1. What is the Task - Define the task you want to achieve from the model
    2. Whats the Role - Define the role model should act like to accomplish that task
    3. Whats the Response Style - Define the response structure or style that should be followed based on the consumer of the output. 
    4. What set of Instructions to be followed:  Define the  set of instructions that model should follow to respond as per the success criteria
2. **Success Criteria**: Clearly define the success criteria or evaluation criteria. This can be in the form of a list of bullet points or as specific as some evaluation metrics (Eg: Length checks, BLEU Score, Rouge, Format, Factuality, Faithfulness). The success criteria should clearly reflect the definition of "good" that will determine the success of the use case.
3. **Draft Prompt**: Finally, a draft prompt is necessary to initiate the iterative process of prompt engineering.



In [3]:
# read in variables and config from previous notebook
%store -r

In [6]:
from IPython.display import display, Markdown
import base64
import boto3
import json

boto3.setup_default_session(region_name=region_name)
client = boto3.client("bedrock-runtime")


def call_nova(
    model,
    messages,
    system_message="",
    streaming=False,
    max_tokens=512,
    temp=0.7,
    top_p=0.99,
    top_k=20,
    tools=None,
    stop_sequences=[],
    verbose=False,
):
    system_list = [{"text": system_message}]
    inf_params = {
        "max_new_tokens": max_tokens,
        "top_p": top_p,
        "top_k": top_k,
        "temperature": temp,
        "stopSequences": stop_sequences,
    }
    request_body = {
        "messages": messages,
        "system": system_list,
        "inferenceConfig": inf_params,
    }
    if tools is not None:
        tool_config = []
        for tool in tools:
            tool_config.append({"toolSpec": tool})
        request_body["toolConfig"] = {"tools": tool_config}
    if verbose:
        print("Request Body", request_body)
    if not streaming:
        response = client.invoke_model(modelId=model, body=json.dumps(request_body))
        model_response = json.loads(response["body"].read())
        return model_response, model_response["output"]["message"]["content"][0]["text"]
    else:
        response = client.invoke_model_with_response_stream(
            modelId=model, body=json.dumps(request_body)
        )
        return response["body"]


def get_base64_encoded_value(media_path):
    with open(media_path, "rb") as media_file:
        binary_data = media_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode("utf-8")
        return base64_string


def print_output(content_text):
    display(Markdown(content_text))


def validate_json(json_string):
    try:
        # Attempt to parse the JSON string
        parsed_json = json.loads(json_string)

        # If successful, return the parsed JSON
        print("Valid JSON")
        return parsed_json

    except json.JSONDecodeError as e:
        # If parsing fails, print an error message
        print(f"Invalid JSON: {e}")

        # Optionally, you can print the location of the error
        print(f"Error at line {e.lineno}, column {e.colno}")

        # Return None to indicate failure
        return None

### Structured Outputs

Sometimes it is essential to make sure that the model only responds in a specific output schema that works best for the downstream use cases (for example, automated workflows where inputs and outputs must always be in a specific format). Amazon Nova models can be instructed to generate responses in a structured way. For example, if the downstream parser expects certain naming of keys in the JSON object, specifying them in an output schema field in your query yields the model to respect that schema. If the need is to be able to parse the schema directly without any preamble, the model can be instructed to output only JSON by saying “Please generate only the JSON output. DO NOT provide any preamble.” at the end of your query. 

#### Using Prefill to Help the Model Get Started

An alternate technique to achieve this efficiently is to nudge the model response via prefilling the assistant content. This technique enables the user to direct the model's actions (putting words in the model's mouth), bypass preambles, and enforce specific output formats such as JSON or XML. For example, by prefilling assistant content with “{” or ```json, you can guide model to skip generating any preamble text and start generating JSON object right away. 


➡️ If the user is explicitly looking for extracting JSON, one common observed pattern is to prefill it with ``json and add a stop sequence on ```, this ensures that the model outputs a JSON object that can be programmatically parsed.


In [7]:
unoptimized_prompt = """Provide details about the best selling full-frame cameras in past three years.
Answer in JSON format with keys like name, brand, price and a summary.
"""

messages = [
    {"role": "user", "content": [{"text": unoptimized_prompt}]},
]

In [8]:
model_response, content_text = call_nova(LITE_MODEL_ID, messages)
print("\n[Response Content Text]")
print("-" * 40)
print(content_text)
print("-" * 40)


[Response Content Text]
----------------------------------------
```json
{
    "rows": [
        {
            "Name": "Canon EOS R5",
            "Brand": "Canon",
            "Price": "$3,899",
            "Summary": "The Canon EOS R5 is a full-frame mirrorless camera with a 45-megapixel sensor, 8K video recording, and advanced autofocus capabilities. It is a top choice for professional photographers and videographers."
        },
        {
            "Name": "Sony Alpha A7R IV",
            "Brand": "Sony",
            "Price": "$3,499",
            "Summary": "The Sony Alpha A7R IV is a full-frame mirrorless camera with a 61-megapixel sensor, 4K video recording, and advanced autofocus capabilities. It is a popular choice for professional photographers and videographers."
        },
        {
            "Name": "Nikon Z7 II",
            "Brand": "Nikon",
            "Price": "$2,999",
            "Summary": "The Nikon Z7 II is a full-frame mirrorless camera with a 45.7-megapixel

### Lets add more schema definition with the right data types and use Prefill

In [9]:
optimized_prompt = """Provide 5 examples of the best selling full-frame cameras in past three years.
Follow the Output Schema as described below:
Output Schema:
{
"name" : <string, the name of product>,
"brand" : <string, the name of product>,
"price" : <integer price>,
"summary": <string, the product summary>
}
Only Respond in Valid JSON, without Markdown
"""
messages = [
    {"role": "user", "content": [{"text": optimized_prompt}]},
    {"role": "assistant", "content": [{"text": "```json"}]},
]

model_response, content_text = call_nova(LITE_MODEL_ID, messages, stop_sequences=["]"])
print("\n[Response Content Text]")
print("-" * 40)
print(content_text)
print("-" * 40)

print("Testing valid JSON:")
parsed_json = validate_json(content_text)
if parsed_json:
    print(parsed_json)


[Response Content Text]
----------------------------------------

[
    {
        "name": "Canon EOS R5",
        "brand": "Canon",
        "price": 3899,
        "summary": "The Canon EOS R5 is a high-resolution full-frame mirrorless camera featuring a 45MP sensor, 8K video recording, and advanced autofocus capabilities."
    },
    {
        "name": "Sony Alpha A7R IV",
        "brand": "Sony",
        "price": 3999,
        "summary": "The Sony Alpha A7R IV is a top-tier full-frame mirrorless camera with a 61MP sensor, 4K video, and exceptional low-light performance."
    },
    {
        "name": "Nikon Z7 II",
        "brand": "Nikon",
        "price": 2999,
        "summary": "The Nikon Z7 II is a professional full-frame mirrorless camera with a 45.7MP sensor, dual EXPEED 6 processors, and excellent image quality."
    },
    {
        "name": "Canon EOS R6",
        "brand": "Canon",
        "price": 2499,
        "summary": "The Canon EOS R6 is a versatile full-frame mirrorless

### Few Shot Example

Including a few examples of the task within your prompt can help to guide Amazon Nova models to generate responses more aligned with your desired outcome. This technique of providing examples to the model to achieve the desired outcome is called few shot prompting. By including the examples using a structured template, you can enable the models to follow instructions, reduce ambiguity, and enhance the accuracy and quality more reliably. This method also helps in clarifying complex instructions or tasks, making it easier for the models to understand and interpret what is being asked. 

**How adding examples to the prompt help**:
Adding examples can help the model with producing 

* Consistent responses which are uniform to the style of the examples 
* Performant responses due to reducing the chance of misinterpreting instructions, and minimizing hallucinations


**Characteristics of Good Shots in prompt**:
The amount by which model performance improves using few shot prompting will depend on the quality and diversity of your chosen examples. 

* **Select diverse examples**: The examples chosen should represent the distribution of your expected input/output in terms of diversity (ranging from common use cases to edge cases) to adequately cover relevant use cases. It is important to avoid any biases in your examples, as bias in the inputs can cause outputs to be biased as well.
* **Match complexity levels**: The complexity of the examples provided should align with the target task or scenario. It is important to make sure the complexity grade is mapped between expected the input and the chosen example in the prompt.
* **Ensure relevance**: The examples selected should be directly relevant to the problem or objective at hand. This ensures consistency and uniformity in responses. 

➡️ Tip: If the above suggestions not work, it is also recommended to build a RAG-based system that augments the prompt with dynamic selection of shots based on the similarities between a user-input query and an available pool of shots.


In [10]:
no_shot = """Your task is to Classify the following texts into the appropriate sentiment classes. The categories to classify are:

Sentiment Classes:
- Positive
- Negative
- Neutral

Query:
Input: The movie makes users think about their lives with the teenagers while still making audience unclear on the storyline.

"""

messages = [{"role": "user", "content": [{"text": no_shot}]}]

model_response, content_text = call_nova(LITE_MODEL_ID, messages)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)


[Response Content Text]
----------------------------------------


To classify the given text, we need to analyze the sentiment expressed in the text. Here's the step-by-step process:

1. Read the text: "The movie makes users think about their lives with the teenagers while still making audience unclear on the storyline."

2. Identify the sentiment expressed in the text:
   - The text mentions that the movie makes users think about their lives with the teenagers, which could be considered a positive aspect.
   - However, the text also mentions that the audience is unclear on the storyline, which is a negative aspect.

3. Determine the overall sentiment based on the identified aspects:
   - The text has both positive and negative aspects, but neither of them is strongly expressed. The text seems to be more focused on the movie's ability to make users think about their lives with the teenagers, which is a positive aspect, but it also highlights the confusion about the storyline, which is a negative aspect.

4. Classify the text into the appropriate sentiment class:
   - Since the text has both positive and negative aspects, and neither of them is strongly expressed, the overall sentiment of the text can be considered as Neutral.

Output: Neutral

----------------------------------------


As you can see there is too much explaination not even asked for, this makes parsing a bit tricky 

Now lets try adding four shots that conveys the meaning and also forces a more stylistic edit on the response 

### With adding few shot prompting

In [ ]:
four_shot = """Your task is to Classify the following texts into the appropriate sentiment classes. The categories to classify are:

Sentiment Classes:
- Positive
- Negative
- Neutral

Please refer to some examples mentioned below.

## Examples
### Example 1
Input: The movie was crazy good! I loved it
Output: Positive
Explaination: The text said "good" and "loved" so its positive

### Example 2
Input: The movie was scary and I got scared!
Output: Neutral
Explaination: The text said "scary" and "scared" which can be both positive and negative depending on people who like scary movies or one who hate

### Example 3
Input: The movie was pathetic not worth the time or money!
Output: Negative
Explaination: The text said "pathetic" and "not worth" which is negative sentiment

### Example 4
Input: The movie had some plots which were interesting and great while there were some gaps which needed more drama!
Output: Neutral
Explaination: The text said "interesting and great" and "some gaps" making it a mixed opinion hence neutral

Query:
Input: The movie makes users think about their lives with the teenagers while still making audience unclear on the storyline.
"""


messages = [{"role": "user", "content": [{"text": four_shot}]}]

model_response, content_text = call_nova(LITE_MODEL_ID, messages)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)

In [ ]:
no_cot = """You are a project manager for a small software development team tasked with launching a new app feature.
You want to streamline the development process and ensure timely delivery. Draft a project plan
"""
messages = [{"role": "user", "content": [{"text": no_cot}]}]

model_response, content_text = call_nova(LITE_MODEL_ID, messages, max_tokens=1024)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)

The above is great but its too long, and you see it keeps going on more. As mentioned this is for Executives, do we want to keep it this long?


Lets now give some guiding questions for model to come up with a draft but first do the thinking


### With Guided Chain of Thought

In [11]:
guided_cot = """You are a project manager for a small software development team tasked with launching a new app feature.
You want to streamline the development process and ensure timely delivery.
Your task is to draft a project plan.

But first do some thinking on how you want to structure and go through below questions before starting the draft.
Please follow these steps:
1. Think about who the audience is (this is for CEOs, CTOs and other executives)
2. Think about what to start with
3. Think about what Challenges you want to solve with this app
4. Think about the Tasks that will be needed to be completed
5. Create Milestones
6. Monitor Progress and Optimize
Explain all your thinking in <thinking></thinking> XML Tags and then write the final copy of project plan for executives in <project_plan></project_plan> XML Tag.

Output Schema:
<thinking>
( thoughts to above questions)
</thinking>
<project_plan>
( project plan)
</project_plan>
"""
messages = [{"role": "user", "content": [{"text": guided_cot}]}]

model_response, content_text = call_nova(LITE_MODEL_ID, messages, max_tokens=2048)
print("\n[Response Content Text]")
print("-" * 40)
print_output(content_text)
print("-" * 40)


[Response Content Text]
----------------------------------------


<thinking>
1. **Audience**: The audience for this project plan includes CEOs, CTOs, and other executives. They need a high-level overview that highlights the strategic importance, potential risks, and expected outcomes. The language should be clear, concise, and focused on the business value.

2. **What to Start With**: The project plan should start with a clear statement of the project's objectives and its alignment with the company's strategic goals. This sets the context and importance of the project.

3. **Challenges to Solve**: The primary challenges to address include ensuring timely delivery of the new app feature, maintaining high-quality standards, managing resources efficiently, and mitigating potential risks. Additionally, we need to ensure that the feature meets user needs and integrates seamlessly with existing systems.

4. **Tasks Needed**: The tasks will include requirement gathering, design, development, testing, deployment, and post-launch support. Each task will be broken down into sub-tasks to ensure detailed planning and accountability.

5. **Milestones**: Key milestones will mark the completion of major phases such as project initiation, design approval, development completion, testing, and launch. These milestones will help in tracking progress and ensuring timely delivery.

6. **Monitor Progress and Optimize**: Regular progress reviews, performance metrics, and feedback loops will be established to monitor progress and optimize the development process. This includes sprint reviews, retrospectives, and performance dashboards.
</thinking>
<project_plan>
<project_title>Project Plan for New App Feature Launch</project_title>
<project_objective>
To launch a new app feature that enhances user experience, meets business objectives, and is delivered on time and within budget.
</project_objective>
<project_scope>
The project involves the development, testing, and deployment of a new app feature. It includes requirement gathering, design, development, testing, and post-launch support.
</project_scope>
<challenges>
- Ensuring timely delivery of the new app feature.
- Maintaining high-quality standards.
- Efficient resource management.
- Mitigating potential risks.
- Ensuring the feature meets user needs and integrates seamlessly with existing systems.
</challenges>
<tasks>
1. Requirement Gathering
   - Stakeholder meetings
   - User interviews
   - Documentation of requirements

2. Design
   - UI/UX design
   - Architecture design
   - Design approval

3. Development
   - Front-end development
   - Back-end development
   - Integration with existing systems

4. Testing
   - Unit testing
   - Integration testing
   - User acceptance testing (UAT)

5. Deployment
   - Staging deployment
   - Production deployment
   - Monitoring post-launch

6. Post-Launch Support
   - Bug fixes
   - Performance monitoring
   - User feedback collection
</tasks>
<milestones>
1. Project Initiation
   - Completion of requirement gathering
   - Approval of project scope and objectives

2. Design Approval
   - Finalization of UI/UX design
   - Approval of architecture design

3. Development Completion
   - Completion of front-end development
   - Completion of back-end development

4. Testing Completion
   - Successful completion of UAT
   - Approval for production deployment

5. Launch
   - Successful deployment to production
   - Post-launch monitoring and support
</milestones>
<monitoring_and_optimization>
- Regular progress reviews every two weeks.
- Performance metrics to track development velocity, defect rates, and user satisfaction.
- Sprint retrospectives to identify areas for improvement.
- Feedback loops with stakeholders to ensure alignment with business objectives.
- Performance dashboards to visualize key metrics and progress.
</monitoring_and_optimization>
</project_plan>

----------------------------------------
